# Maximum Likelihood Estimation (MLE)
## 최대 가능도 추정
### 이미 관측된 데이터(결과)가 나올 확률이 가장 높은 파라미터(원인)를 역으로 추적해서 찾아내는 방법 
ex) 동전을 열 번 던져 앞면이 9번 나왔다면, 해당 동전의 앞면이 나올 확률은 90% 짜리 동전이라고 가장 그럴듯한 수치를 추정하는 것. 

### 베르누이 분포 (binomial) : 두 가지 클래스
$$
\begin{aligned}
K &\sim \mathcal{B}(n, \theta) \\[1em]
P(K = k) &= \binom{n}{k} \theta^{k} (1-\theta)^{n-k} \\
&= \frac{n!}{k!(n-k)!} \cdot \theta^{k} (1-\theta)^{n-k}
\end{aligned}
$$

n = 100, k = 27이라는 observation을 했을 때 P에 해당 값을 넣어 확률을 결정하는 세타, 파라미터 값을 알고 싶어 함.
- 공식에 따라 f(세타)를 세타에 따라 값을 다르게 구할 수 있음. 
- observation을 가장 잘 설명하는 theta를 찾아내기 = 확률이 제일 높은 경우의 세타를 찾아내기

### optimization via Gradient Descent
그래프에서 가장 최고점을 구하기 위해서 Gradient Descent를 사용해 움직이기 가능. 

## Overfitting
단 한 번의 실험 결과($k=27$)를 절대적인 진리로 과신한 나머지, 그것이 우연히 발생한 오차(Noise)일 가능성을 무시하고 일반적인 법칙으로 단정 짓는 것
- validation set과 train set 사이 score 격차가 커짐 : 새로운 데이터에는 전혀 통하지 않음

방지법
- more data
- less features
- regularization

## Regularization : 오버피팅 방지
- early stopping : val loss가 더 이상 낮아지지 않을 때
- reducing network size : 뉴럴 네트워크가 학습할 수 있는 양을 줄이기
- weight decay : weight parameter 크기 줄이기
- dropout
- batch normalization

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [ ]:
x_train = torch.FloatTensor([[1, 2, 1],
                             [1, 3, 2],
                             [1, 3, 4],
                             [1, 5, 5],
                             [1, 7, 5],
                             [1, 2, 5],
                             [1, 6, 6],
                             [1, 7, 7]
                            ])
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0])

# x train 크기 (m, 3)
# y 크기 (m,)

In [12]:
x_test = torch.FloatTensor([[2, 1, 1], [3, 1, 2], [3, 3, 4]])
y_test = torch.LongTensor([2, 2, 2])

# Model

In [ ]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 3)
    def forward(self, x):
        return self.linear(x) # x 사이즈 (m, 3)

In [4]:
model = SoftmaxClassifierModel()

In [5]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Training

In [ ]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):
        # H(x) 계산
        prediction = model(x_train) # x_train 사이즈 : (m, 3)
                                    # prediction 사이즈 : (m, 3)

        # cost 계산
        cost = F.cross_entropy(prediction, y_train)
        # y_train 사이즈 : (m,)

        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

# Test (Validation)

In [9]:
def test(model, optimizer, x_test, y_test):
    prediction = model(x_test)
    predicted_classes = prediction.max(1)[1]

    # .max(1)[1] : prediction 값 중에서 가장 큰 값 찾음
    # (가장 큰 점수, 그 점수의 인덱스) 중 두 번째 인덱스만 가져 옴.
    correct_count = (predicted_classes == y_test).sum().item()
    cost = F.cross_entropy(prediction, y_test)

    print('Accuracy: {}% Cost: {:.6f}'.format(correct_count / len(y_test) * 100, cost.item()))

# Run

In [10]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 2.203667
Epoch    1/20 Cost: 1.199645
Epoch    2/20 Cost: 1.142985
Epoch    3/20 Cost: 1.117769
Epoch    4/20 Cost: 1.100901
Epoch    5/20 Cost: 1.089523
Epoch    6/20 Cost: 1.079872
Epoch    7/20 Cost: 1.071320
Epoch    8/20 Cost: 1.063325
Epoch    9/20 Cost: 1.055720
Epoch   10/20 Cost: 1.048378
Epoch   11/20 Cost: 1.041245
Epoch   12/20 Cost: 1.034285
Epoch   13/20 Cost: 1.027478
Epoch   14/20 Cost: 1.020813
Epoch   15/20 Cost: 1.014279
Epoch   16/20 Cost: 1.007872
Epoch   17/20 Cost: 1.001586
Epoch   18/20 Cost: 0.995419
Epoch   19/20 Cost: 0.989365


In [13]:
test(model, optimizer, x_test, y_test)

Accuracy: 0.0% Cost: 1.425844


# Learning Rate
lr이 너무 크면 수렴 하면서 cost가 점점 늘어난다 = overshooting!

In [14]:
model = SoftmaxClassifierModel()
optimizer = optim.SGD(model.parameters(), lr=1e5)
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 1.280268
Epoch    1/20 Cost: 976950.687500
Epoch    2/20 Cost: 1279135.125000
Epoch    3/20 Cost: 1198379.125000
Epoch    4/20 Cost: 1098825.625000
Epoch    5/20 Cost: 1968197.625000
Epoch    6/20 Cost: 284763.156250
Epoch    7/20 Cost: 1532260.125000
Epoch    8/20 Cost: 1651504.000000
Epoch    9/20 Cost: 521878.437500
Epoch   10/20 Cost: 1397263.125000
Epoch   11/20 Cost: 750986.250000
Epoch   12/20 Cost: 918691.500000
Epoch   13/20 Cost: 1487888.125000
Epoch   14/20 Cost: 1582260.125000
Epoch   15/20 Cost: 685818.062500
Epoch   16/20 Cost: 1140048.750000
Epoch   17/20 Cost: 940566.562500
Epoch   18/20 Cost: 931638.125000
Epoch   19/20 Cost: 1971322.625000


lr이 너무 작으면 cost가 거의 줄어들지 않는다.

In [15]:
model = SoftmaxClassifierModel()
optimizer = optim.SGD(model.parameters(), lr=1e-10)
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 3.187324
Epoch    1/20 Cost: 3.187324
Epoch    2/20 Cost: 3.187324
Epoch    3/20 Cost: 3.187324
Epoch    4/20 Cost: 3.187324
Epoch    5/20 Cost: 3.187324
Epoch    6/20 Cost: 3.187324
Epoch    7/20 Cost: 3.187324
Epoch    8/20 Cost: 3.187324
Epoch    9/20 Cost: 3.187324
Epoch   10/20 Cost: 3.187324
Epoch   11/20 Cost: 3.187324
Epoch   12/20 Cost: 3.187324
Epoch   13/20 Cost: 3.187324
Epoch   14/20 Cost: 3.187324
Epoch   15/20 Cost: 3.187324
Epoch   16/20 Cost: 3.187324
Epoch   17/20 Cost: 3.187324
Epoch   18/20 Cost: 3.187324
Epoch   19/20 Cost: 3.187324


적절한 lr을 적용해야 함.
발산하면 작게, 줄어들지 않으면 크게 조정하기. 

In [16]:
model = SoftmaxClassifierModel()
optimizer = optim.SGD(model.parameters(), lr=1e-1)
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 1.341574
Epoch    1/20 Cost: 1.198802
Epoch    2/20 Cost: 1.150877
Epoch    3/20 Cost: 1.131978
Epoch    4/20 Cost: 1.116242
Epoch    5/20 Cost: 1.102514
Epoch    6/20 Cost: 1.089676
Epoch    7/20 Cost: 1.077479
Epoch    8/20 Cost: 1.065775
Epoch    9/20 Cost: 1.054511
Epoch   10/20 Cost: 1.043655
Epoch   11/20 Cost: 1.033187
Epoch   12/20 Cost: 1.023091
Epoch   13/20 Cost: 1.013356
Epoch   14/20 Cost: 1.003968
Epoch   15/20 Cost: 0.994917
Epoch   16/20 Cost: 0.986189
Epoch   17/20 Cost: 0.977775
Epoch   18/20 Cost: 0.969661
Epoch   19/20 Cost: 0.961836


# Data Preprocessing

In [17]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90], 
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

$$
x'_j = \frac{x_j - \mu_j}{\sigma_j}
$$

$\sigma$는 standard derivation(표준 편차), $\mu$는 평균값.

In [18]:
mu = x_train.mean(dim=0)
sigma = x_train.std(dim=0)
norm_x_train = (x_train - mu)/sigma
print(norm_x_train)

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])


# Training with Preprocessed Data

In [19]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

In [20]:
model = MultivariateLinearRegressionModel()
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [21]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs + 1):
        prediction = model(x_train)
        cost = F.mse_loss(prediction, y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

In [22]:
train(model, optimizer, norm_x_train, y_train)

Epoch    0/20 Cost: 29785.089844
Epoch    1/20 Cost: 18906.166016
Epoch    2/20 Cost: 12054.675781
Epoch    3/20 Cost: 7702.029785
Epoch    4/20 Cost: 4925.733398
Epoch    5/20 Cost: 3151.632812
Epoch    6/20 Cost: 2016.996094
Epoch    7/20 Cost: 1291.051270
Epoch    8/20 Cost: 826.505249
Epoch    9/20 Cost: 529.207397
Epoch   10/20 Cost: 338.934174
Epoch   11/20 Cost: 217.153564
Epoch   12/20 Cost: 139.206757
Epoch   13/20 Cost: 89.313782
Epoch   14/20 Cost: 57.375465
Epoch   15/20 Cost: 36.928429
Epoch   16/20 Cost: 23.835773
Epoch   17/20 Cost: 15.450401
Epoch   18/20 Cost: 10.077823
Epoch   19/20 Cost: 6.633700
Epoch   20/20 Cost: 4.423972
